# 01a - Visualizing the sites

This notebook reproduces Fig. S2 from the manuscript, which visualizes the adsorption sites in the supercell. It also demonstrates how to generate the sites, and estimate the number of combinations (Tables S1 and S2).

In [1]:
import os
import math
import numpy as np
import pandas as pd
from pymatgen.core import Structure, Molecule
from mkite_catalysis.runners.coverage import CoverageGenerator

## Loading the basics

In [2]:
helium = Molecule(species=["He"], coords=[[0, 0, 0]])

In [3]:
facet_dir = "../data/01-facets"
facets = [100, 111, 211, 331, 410, 711]

cutoffs = {
    111: 1.0,
    100: 1.0,
    211: 1.4,
    331: 1.6,
    410: 1.2,
    711: 1.8
}

num_sites = {100: 16, 111: 16, 211: 24, 331: 24, 410: 24, 711: 16}

surfaces = {}
for facet in facets:
    fname = os.path.join(facet_dir, f"{facet}.POSCAR")
    surfaces[facet] = Structure.from_file(fname)

## Making the sites

In [4]:
for facet, surf in surfaces.items():
    covgen = CoverageGenerator(
        surf,
        helium,
        surface_height=cutoffs[facet],
        adsorption_height=2,
        distance_threshold=2,
    )

    sites = covgen.get_sites()

    new = surf.copy()

    for xyz in sites:
        new.append(
            species="He",
            coords=xyz,
            coords_are_cartesian=True,
            properties={"selective_dynamics": [False, False, False]}
        )
    
    new.to(f"../data/01-facets-sites/{facet}.cif")

/opt/homebrew/Caskroom/miniconda/base/envs/mkite/lib/python3.9/site-packages/pymatgen/core/periodic_table.py:265: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


## Counting the sites

In [5]:
site_stats = {}

for facet, surf in surfaces.items():
    covgen = CoverageGenerator(
        surf,
        helium,
        surface_height=cutoffs[facet],
        adsorption_height=2,
        distance_threshold=2,
    )

    sites = covgen.get_sites()
    
    site_stats[facet] = {
        "a": surf.lattice.a,
        "b": surf.lattice.b,
        "n_sites": len(sites),
        "surf_depth": cutoffs[facet],
        "surf_atoms": num_sites[facet],
    }
    
df = pd.DataFrame(site_stats).T

In [6]:
table = df.copy()

for col in ["n_sites", "surf_atoms"]:
    table[col] = table[col].astype(int)

In [7]:
table = table[["a", "b", "surf_depth", "surf_atoms", "n_sites"]]

In [8]:
print(table.to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
 & a & b & surf_depth & surf_atoms & n_sites \\
\midrule
100 & 10.39 & 10.39 & 1.00 & 16 & 64 \\
111 & 10.39 & 9.00 & 1.00 & 16 & 96 \\
211 & 10.39 & 12.73 & 1.40 & 24 & 128 \\
331 & 10.39 & 11.32 & 1.60 & 24 & 144 \\
410 & 11.02 & 15.15 & 1.20 & 24 & 120 \\
711 & 10.39 & 9.28 & 1.80 & 16 & 64 \\
\bottomrule
\end{tabular}



## Counting combinations of sites

In [9]:
n_sites = (df["n_sites"] / 4).astype(int)

In [10]:
n_sites

100    16
111    24
211    32
331    36
410    30
711    16
Name: n_sites, dtype: int64

In [11]:
combos = {}
for k in range(1, 25):
    comb_n = {}
    for facet, n in n_sites.items():
        if k > n:
            continue
        
        comb_n[facet] = math.comb(n, k)
    
    combos[k] = comb_n

combos = pd.DataFrame(combos).T

In [12]:
combos = combos.fillna(0).astype(int)

In [13]:
def simplify_notation(n):
    if n < 1e4:
        return str(n)
    
    return f"{n:.2e}"

In [14]:
for col in combos.columns:
    combos[col] = combos[col].apply(simplify_notation)

In [15]:
combos

,100,111,211,331,410,711
1,16,24,32,36,30,16
2,120,276,496,630,435,120
3,560,2024,4960,7140,4060,560
4,1820,1.06e+04,3.60e+04,5.89e+04,2.74e+04,1820
5,4368,4.25e+04,2.01e+05,3.77e+05,1.43e+05,4368
6,8008,1.35e+05,9.06e+05,1.95e+06,5.94e+05,8008
7,1.14e+04,3.46e+05,3.37e+06,8.35e+06,2.04e+06,1.14e+04
8,1.29e+04,7.35e+05,1.05e+07,3.03e+07,5.85e+06,1.29e+04
9,1.14e+04,1.31e+06,2.80e+07,9.41e+07,1.43e+07,1.14e+04
10,8008,1.96e+06,6.45e+07,2.54e+08,3.00e+07,8008


In [16]:
print(combos.to_latex())

\begin{tabular}{lllllll}
\toprule
 & 100 & 111 & 211 & 331 & 410 & 711 \\
\midrule
1 & 16 & 24 & 32 & 36 & 30 & 16 \\
2 & 120 & 276 & 496 & 630 & 435 & 120 \\
3 & 560 & 2024 & 4960 & 7140 & 4060 & 560 \\
4 & 1820 & 1.06e+04 & 3.60e+04 & 5.89e+04 & 2.74e+04 & 1820 \\
5 & 4368 & 4.25e+04 & 2.01e+05 & 3.77e+05 & 1.43e+05 & 4368 \\
6 & 8008 & 1.35e+05 & 9.06e+05 & 1.95e+06 & 5.94e+05 & 8008 \\
7 & 1.14e+04 & 3.46e+05 & 3.37e+06 & 8.35e+06 & 2.04e+06 & 1.14e+04 \\
8 & 1.29e+04 & 7.35e+05 & 1.05e+07 & 3.03e+07 & 5.85e+06 & 1.29e+04 \\
9 & 1.14e+04 & 1.31e+06 & 2.80e+07 & 9.41e+07 & 1.43e+07 & 1.14e+04 \\
10 & 8008 & 1.96e+06 & 6.45e+07 & 2.54e+08 & 3.00e+07 & 8008 \\
11 & 4368 & 2.50e+06 & 1.29e+08 & 6.01e+08 & 5.46e+07 & 4368 \\
12 & 1820 & 2.70e+06 & 2.26e+08 & 1.25e+09 & 8.65e+07 & 1820 \\
13 & 560 & 2.50e+06 & 3.47e+08 & 2.31e+09 & 1.20e+08 & 560 \\
14 & 120 & 1.96e+06 & 4.71e+08 & 3.80e+09 & 1.45e+08 & 120 \\
15 & 16 & 1.31e+06 & 5.66e+08 & 5.57e+09 & 1.55e+08 & 16 \\
16 & 1 & 7.35e+05 